# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
path = "../output_data/the_cities.csv"
vacation_df = pd.read_csv(path)
vacation_df

,City,Lat,Long,Temp (C),Humidity,Cloudiness,Wind Speed,Country,GMT
0,Khatanga,71.97,102.50,-13.99,86,12,3.90,RU,1603043205
1,Tiksi,71.69,128.87,-7.52,96,100,8.05,RU,1603043205
2,Castro,-24.79,-50.01,20.33,74,100,3.57,BR,1603043205
3,Rodrigues Alves,-7.74,-72.65,35.41,37,69,1.51,BR,1603043205
4,Punta Arenas,-53.15,-70.92,8.00,65,75,10.80,CL,1603043072
...,...,...,...,...,...,...,...,...,...
563,Puerto Carreño,6.19,-67.49,32.49,54,48,0.48,CO,1603043003
564,Pocatello,42.87,-112.45,13.89,24,40,11.80,US,1603043002
565,Bayburt,40.26,40.23,12.17,36,17,1.30,TR,1603043259
566,Quatre Cocos,-20.21,57.76,24.00,69,40,3.60,MU,1603043259


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#gmaps
gmaps.configure(api_key=g_key)

locations = vacation_df[["Lat", "Long"]]
weight = vacation_df["Humidity"]

humidity_heatmap = gmaps.figure(map_type='SATELLITE')
humidity_heatmap.add_layer(gmaps.heatmap_layer(locations, weights=weight))
humidity_heatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [20]:
hotel_df = vacation_df.loc[(vacation_df["Temp (C)"] > 22) & (vacation_df["Wind Speed"] < 10) & 
                           (vacation_df["Cloudiness"] == 0)]

hotel_df




,City,Lat,Long,Temp (C),Humidity,Cloudiness,Wind Speed,Country,GMT,Hotel Name
37,Chui,-33.70,-53.46,25.48,53,0,3.36,UY,1603043208,
47,Kidal,18.44,1.41,34.19,11,0,1.48,ML,1603043209,
169,Alexandria,31.22,29.96,24.00,73,0,3.10,EG,1603043221,
178,Poum,-20.23,164.02,22.80,74,0,7.96,NC,1603043222,
179,Kiryat Gat,31.61,34.76,26.00,74,0,2.60,IL,1603043222,
188,San Luis,-33.30,-66.34,35.91,18,0,5.54,AR,1603043223,
204,Bhabhua,25.05,83.62,23.95,57,0,1.33,IN,1603043225,
232,Bilma,18.69,12.92,31.60,11,0,3.07,NE,1603043227,
278,General Roca,-39.03,-67.58,23.00,23,0,2.60,AR,1603043232,
280,Santa Maria,-29.68,-53.81,30.71,41,0,1.58,BR,1603043232,


In [21]:
hotel_name = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

# taken from Airpost_Ratings Example done in class
for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    lng = row["Long"]

#put lat and long in params   
    params["location"] = f"{lat},{lng}"

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

#make request for json stuff
    name_address = requests.get(base_url, params=params)
    
#convert to json
    name_address = name_address.json()
    
    try:
#         hotel_df.iloc[index, 4] = name_address['results'][0]['name']
        hotel_name.append(name_address['results'][0]['name'])
    except IndexError:
        hotel_name.append(np.nan)

In [22]:
hotel_df["Hotel Name"] = hotel_name
hotel_df

/Users/tylerbrown/opt/anaconda3/envs/TestEnvironment/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,City,Lat,Long,Temp (C),Humidity,Cloudiness,Wind Speed,Country,GMT,Hotel Name
37,Chui,-33.70,-53.46,25.48,53,0,3.36,UY,1603043208,Bertelli Chuí Hotel
47,Kidal,18.44,1.41,34.19,11,0,1.48,ML,1603043209,NaN
169,Alexandria,31.22,29.96,24.00,73,0,3.10,EG,1603043221,Miramar Boutique hotel
178,Poum,-20.23,164.02,22.80,74,0,7.96,NC,1603043222,NaN
179,Kiryat Gat,31.61,34.76,26.00,74,0,2.60,IL,1603043222,Desert Gat
188,San Luis,-33.30,-66.34,35.91,18,0,5.54,AR,1603043223,GRAN HOTEL SAN LUIS GOLD
204,Bhabhua,25.05,83.62,23.95,57,0,1.33,IN,1603043225,Hotel Sivoy
232,Bilma,18.69,12.92,31.60,11,0,3.07,NE,1603043227,NaN
278,General Roca,-39.03,-67.58,23.00,23,0,2.60,AR,1603043232,Hotel El Recreo
280,Santa Maria,-29.68,-53.81,30.71,41,0,1.58,BR,1603043232,Itaimbé Palace Hotel


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [14]:
locations2 = hotel_df[["Lat", "Long"]]

hotel_layer = gmaps.symbol_layer(
    locations2, fill_color='blue', stroke_color='blue', scale=2
)

humidity_heatmap.add_layer(hotel_layer)
humidity_heatmap

Figure(layout=FigureLayout(height='420px'))

In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations3 = hotel_df[["Lat", "Long"]]

In [26]:
# Add marker layer ontop of heat map


hotel_layer2 = gmaps.symbol_layer(
    locations3, fill_color='green', stroke_color='green', scale=2
)

humidity_heatmap.add_layer(hotel_layer2)
humidity_heatmap
# Display figure


Figure(layout=FigureLayout(height='420px'))